In [1]:
# Classics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Base
from collections import Counter
import re
import os

# Plotting
import squarify
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

# Vector Representations
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

In [2]:
df = pd.read_csv('./lyrics.csv')
df.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [3]:
df = df.drop(['artist', 'track_n'], axis = 1)

In [4]:
df.head()

,album,track_title,lyric,line,year
0,Taylor Swift,Tim McGraw,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Tim McGraw,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Tim McGraw,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Tim McGraw,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Tim McGraw,That had a tendency of gettin' stuck,5,2006


In [5]:
df['album'].value_counts()

reputation      1006
1989             953
Speak Now        930
Red              845
Taylor Swift     567
Fearless         561
Name: album, dtype: int64

In [6]:
rep = df[df['album'] == 'reputation']
rep['track_title'].unique()

array(['...Ready for It?', 'End Game (Ft.\xa0Ed\xa0Sheeran & Future)',
       'I Did Something Bad', "Don't Blame Me", 'Delicate',
       'Look What You Made Me Do', 'So It Goes...', 'Gorgeous',
       'Getaway Car', 'King of My Heart', 'Dancing With Our Hands Tied',
       'Dress', "This Is Why We Can't Have Nice Things",
       'Call It What You Want', "New Year's Day"], dtype=object)

In [7]:
df = df.replace('&', 'and')
df = df.replace('22', 'Twenty-Two')

## Titles

In [8]:
# Eliminate repetition of titles

titles = pd.DataFrame(df['track_title'])
titles.columns = (['title'])
titles.head()

,title
0,Tim McGraw
1,Tim McGraw
2,Tim McGraw
3,Tim McGraw
4,Tim McGraw


In [9]:
def get_lemmas(text):

    lemmas = []
    
    doc = nlp(text)
    
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['title_lemmas'] = titles['title'].apply(get_lemmas)

In [10]:
# Store NaN-free version in x

x = df['title_lemmas'].dropna()

In [11]:
# Check the average song title length

lengths = []

for item in x:
    number = len(item)
    lengths.append(number)
    
total = sum(lengths)
mean = total / len(x)
mean

1.6624845742492802

In [12]:
def count(docs):

        word_counts = Counter()
        appears_in = Counter()
        
        total_docs = len(docs)

        for doc in docs:
            word_counts.update(doc)
            appears_in.update(set(doc))

        temp = zip(word_counts.keys(), word_counts.values())
        
        wc = pd.DataFrame(temp, columns = ['word', 'count'])

        wc['rank'] = wc['count'].rank(method='first', ascending=False)
        total = wc['count'].sum()

        wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
        wc = wc.sort_values(by='rank')
        wc['cul_pct_total'] = wc['pct_total'].cumsum()

        t2 = zip(appears_in.keys(), appears_in.values())
        ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
        wc = ac.merge(wc, on='word')

        wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
        return wc.sort_values(by='rank')

In [13]:
wc = count(x)
wc_75 = wc[wc['appears_in'] > 75]
wc_top20 = wc[wc['rank'] <= 21]

wc_75

,word,appears_in,count,rank,pct_total,cul_pct_total,appears_in_pct
31,,261,522,1.0,0.064580,0.064580,0.053682
34,Ft,261,261,2.0,0.032290,0.096870,0.053682
12,stay,163,239,3.0,0.029568,0.126438,0.033525
66,know,164,164,4.0,0.020289,0.146728,0.033731
76,Ed,158,158,5.0,0.019547,0.166275,0.032497
77,Sheeran,158,158,6.0,0.019547,0.185822,0.032497
94,bad,125,125,7.0,0.015465,0.201287,0.025710
39,love,121,121,8.0,0.014970,0.216256,0.024887
10,tie,117,117,9.0,0.014475,0.230731,0.024064
86,New,112,112,10.0,0.013856,0.244587,0.023036


In [14]:
# Randomly generate a title based on most common words

import random

words = ['stay', 'know', 'bad', 'love', 'tie', 'new', 'story', 'end', 
        'game', 'future', 'girl', 'change', 'blank', 'space', 'wish',
        'heart', 'enchant', 'delicate', 'beautiful']

def title():
    one = random.choice(words)
    two = random.choice(words)
    
    one = one.title()
    two = two.title()
    
    x = random.randint(0, 10)
    if x > 6:
        print(f'{one} {two}')
    else:
        print(one)
    
title()

Space


## "Tim McGraw" Lyrics

In [15]:
# Let's isolate choruses and verses for each song

first = df[df['track_title'] == 'Tim McGraw']
#first['lyric'].value_counts()

In [16]:
def chorus_verse(df):
    
    chorus = ['When you think Tim McGraw',
          'I hope you think my favorite song',                     
          'I hope you think that little black dress',                 
          'When you think happiness',                                 
          'I hope you think of me',                                   
          'And my old faded blue jeans',                              
          'Think of my head on your chest',                           
          "I said, 'That's a lie'",                                  
          'The moon like a spotlight on the lake',                    
          'Put those Georgia stars to shame that night',              
          'The one we danced to all night long']
    
    values = []
    
    for item in df['lyric']:
        if item in chorus:
            value = 'chorus'
            values.append(value)
        else:
            value = 'verse'
            values.append(value)
            
    return pd.DataFrame(values, columns = ['lyric_type'])

In [17]:
lt = chorus_verse(first)

In [18]:
first = pd.concat([first, lt], axis = 1)
first

,album,track_title,lyric,line,year,title_lemmas,lyric_type
0,Taylor Swift,Tim McGraw,He said the way my blue eyes shined,1,2006,"[Tim, McGraw]",verse
1,Taylor Swift,Tim McGraw,Put those Georgia stars to shame that night,2,2006,"[Tim, McGraw]",chorus
2,Taylor Swift,Tim McGraw,"I said, ""That's a lie""",3,2006,"[Tim, McGraw]",verse
3,Taylor Swift,Tim McGraw,Just a boy in a Chevy truck,4,2006,"[Tim, McGraw]",verse
4,Taylor Swift,Tim McGraw,That had a tendency of gettin' stuck,5,2006,"[Tim, McGraw]",verse
5,Taylor Swift,Tim McGraw,On backroads at night,6,2006,"[Tim, McGraw]",verse
6,Taylor Swift,Tim McGraw,And I was right there beside him all summer long,7,2006,"[Tim, McGraw]",verse
7,Taylor Swift,Tim McGraw,And then the time we woke up to find that summ...,8,2006,"[Tim, McGraw]",verse
8,Taylor Swift,Tim McGraw,But when you think Tim McGraw,9,2006,"[Tim, McGraw]",verse
9,Taylor Swift,Tim McGraw,I hope you think my favorite song,10,2006,"[Tim, McGraw]",chorus


## Let's do that in the whole dataframe

In [19]:
# Identify all songs

songs = []

stuff = df['track_title'].unique()
for item in stuff:
    songs.append(item)

In [20]:
# Sort out which lyrics appear more than once (chorus)

chorus = []

for item in songs:
    temp = df[df['track_title'] == item]
    for val, count in temp['lyric'].value_counts().iteritems():
        if count > 1:
            chorus.append(val)
        else:
            pass

In [21]:
# Label lyrics and either belonging to a verse or chorus

values = []

for item in df['lyric']:
    if item in chorus:
        value = 'chorus'
        values.append(value)
    else:
        value = 'verse'
        values.append(value)

In [22]:
# Add the lyric_type column to our dataframe

lt = pd.DataFrame(values, columns = ['lyric_type'])
final = pd.concat([df, lt], axis = 1)
final

,album,track_title,lyric,line,year,title_lemmas,lyric_type
0,Taylor Swift,Tim McGraw,He said the way my blue eyes shined,1,2006,"[Tim, McGraw]",verse
1,Taylor Swift,Tim McGraw,Put those Georgia stars to shame that night,2,2006,"[Tim, McGraw]",chorus
2,Taylor Swift,Tim McGraw,"I said, ""That's a lie""",3,2006,"[Tim, McGraw]",chorus
3,Taylor Swift,Tim McGraw,Just a boy in a Chevy truck,4,2006,"[Tim, McGraw]",verse
4,Taylor Swift,Tim McGraw,That had a tendency of gettin' stuck,5,2006,"[Tim, McGraw]",verse
...,...,...,...,...,...,...,...
4857,reputation,New Year's Day,"(Hold on to the memories, they will hold on to...",43,2017,"[New, Year, Day]",chorus
4858,reputation,New Year's Day,Please don't ever become a stranger,44,2017,"[New, Year, Day]",chorus
4859,reputation,New Year's Day,"(Hold on to the memories, they will hold on to...",45,2017,"[New, Year, Day]",chorus
4860,reputation,New Year's Day,Whose laugh I could recognize anywhere,46,2017,"[New, Year, Day]",chorus


In [23]:
chorus = final[final['lyric_type'] == 'chorus']
verse = final[final['lyric_type'] == 'verse']

## Let's work with the chorus lyrics

In [24]:
len(chorus['lyric'].unique())

777

In [25]:
len(chorus['track_title'].unique())

94

In [26]:
chorus_length = len(chorus['lyric'].unique()) / len(chorus['track_title'].unique())
chorus_length

8.26595744680851

So it looks like the average chorus has 8 unique lines

## How about the verses?

In [27]:
verse_length = len(verse['lyric'].unique()) / len(verse['track_title'].unique())
verse_length

25.28723404255319

There's a average total of 25 unique verse lyrics. If we go off of songwriting trends, we can assume that this means there are 3 verses, each 8 lines long, with a chorus or partial chorus following each of them

In [28]:
chorus.to_csv('chorus.csv')
verse.to_csv('verse.csv')